In [51]:
from glob import glob
import os
from pathlib import Path
from enum import Enum
import json
import re
import pandas as pd
import seaborn as sns

In [52]:
SYNTHESIS_UTILIZE_UNATE_PATH = "../experiments/synthesis/utilize-unates"
SYNTHESIS_PLAIN_PATH = "../experiments/synthesis/spot-hijack"

# Utils

In [53]:
class Status(Enum):
    ERROR = "Error"
    SUCCESS = "Success"
    NOT_FOUND = "Not Found"
    
    def __repr__(self):
        return str(self.value)

In [54]:
def get_all_benchmarks(path):
    files = glob(os.path.join(path, "*.json"))
    names = [
        Path(f).stem
        for f in files
    ]
    return names
print("Total benchmark of synthesis with dependecy" , len(get_all_benchmarks(SYNTHESIS_UTILIZE_UNATE_PATH)))

Total benchmark of synthesis with dependecy 207


In [55]:
def benchmark_loader(path: str, should_unate: bool):
    def load_benchmark(name: str):
        base = {
            'Name': name,
        }
        
        json_path = os.path.join(path, name + ".json")
        
        if not os.path.exists(json_path):
            return {
                **base,
                'Status': Status.NOT_FOUND
            }
        
        with open(json_path, 'r') as f:
            content = f.readlines()
            stringed_content = "\r\n".join(content)
            try:
                benchmark_json = json.loads(content[1])
            except Exception as e:
                return {
                    **base,
                    'Status': Status.ERROR,
                    'Error': content
                }
        
        if should_unate:
            # print(content)
            assert 'Apply Unates Handling' in content[0]
        else:
            assert 'Skip Unates Handling' in content[0]
        
        is_completed = benchmark_json['is_completed']
        is_automaton_build = benchmark_json['automaton']['is_built']
        
        if not is_automaton_build:
            return {
                **base,
                'Status': Status.TIMEOUT,
                'Is Completed': is_completed,
                'Is Automaton Built': False,
            }
        
        synthesis_duration_pattern = r'=> Synthesis Duration: (?P<duration>\d+)'
        synthesis_duration_match = re.search(synthesis_duration_pattern, stringed_content)
        if synthesis_duration_match:
            synthesis_duration = int(synthesis_duration_match.group('duration'))
        else:
            synthesis_duration = None
        
        automaton_build_duration = benchmark_json['automaton']['build_duration']
        total_states = benchmark_json['automaton']['total_states']
        total_output_vars = len(benchmark_json['output_vars'])

        # Unate handling        
        states_removed_edges = { state: 0 for state in range(total_states) }
        states_impacted_edges = { state: 0 for state in range(total_states) }
        for state_unateness in benchmark_json['unate_states']:
            state = state_unateness['state']
            states_removed_edges[state] = state_unateness['removed_edges']
            states_impacted_edges[state] = state_unateness['impacted_edges']
        
        return {
            **base,
            'Status': Status.SUCCESS,
            'Is Completed': is_completed,
            'Utilized Unate': should_unate,
            
            'Total Impacted Edges': sum(states_impacted_edges.values()),
            'Total Removed Edges': sum(states_removed_edges.values()),
            
            'Is Automaton Built': is_automaton_build,
            'Automaton Build Duration': automaton_build_duration,
            'Total States': total_states,
            'Total Output Vars': total_output_vars,
            
            'Synthesis Duration': synthesis_duration,
            
            'Total Impacted Edges': sum(states_impacted_edges.values()),
            'Total Removed Edges': sum(states_removed_edges.values()),
        }

    return load_benchmark

In [56]:
synthesis_with_unate_loader = benchmark_loader(SYNTHESIS_UTILIZE_UNATE_PATH, should_unate=True)
unate_synthesis = pd.DataFrame([synthesis_with_unate_loader(name) for name in get_all_benchmarks(SYNTHESIS_UTILIZE_UNATE_PATH)])
unate_synthesis.head(5)

,Name,Status,Is Completed,Utilized Unate,Total Impacted Edges,Total Removed Edges,Is Automaton Built,Automaton Build Duration,Total States,Total Output Vars,Synthesis Duration,Error
0,ltl2dpa20,Status.SUCCESS,True,True,0.0,0.0,True,24.0,17.0,2.0,5.0,NaN
1,TwoCountersInRangeA2,Status.SUCCESS,True,True,0.0,0.0,True,338.0,6.0,8.0,178.0,NaN
2,lilydemo24,Status.SUCCESS,True,True,15.0,3.0,True,2.0,16.0,4.0,15.0,NaN
3,OneCounterGuiA9,Status.SUCCESS,True,True,0.0,0.0,True,338.0,9.0,9.0,983.0,NaN
4,ltl2dba24,Status.SUCCESS,True,True,7.0,3.0,True,14.0,6.0,1.0,0.0,NaN


In [57]:
synthesis_without_unate_loader = benchmark_loader(SYNTHESIS_PLAIN_PATH, should_unate=False)
plain_synthesis = pd.DataFrame([synthesis_without_unate_loader(name) for name in get_all_benchmarks(SYNTHESIS_PLAIN_PATH)])
plain_synthesis.head(5)

,Name,Status,Is Completed,Utilized Unate,Total Impacted Edges,Total Removed Edges,Is Automaton Built,Automaton Build Duration,Total States,Total Output Vars,Synthesis Duration,Error
0,ltl2dpa20,Status.SUCCESS,True,False,0.0,0.0,True,4.0,17.0,2.0,17.0,NaN
1,TwoCountersInRangeA2,Status.SUCCESS,True,False,0.0,0.0,True,187.0,6.0,8.0,82.0,NaN
2,lilydemo24,Status.SUCCESS,True,False,0.0,0.0,True,1.0,16.0,4.0,15.0,NaN
3,OneCounterGuiA9,Status.SUCCESS,True,False,0.0,0.0,True,189.0,9.0,9.0,531.0,NaN
4,ltl2dba24,Status.SUCCESS,True,False,0.0,0.0,True,1.0,6.0,1.0,14.0,NaN


In [60]:
all_synthesis = pd.merge(unate_synthesis, plain_synthesis, on='Name', suffixes=('_unate', '_plain'))
all_synthesis.head(5)
all_synthesis.to_csv('./unate_synthesis_compare.csv')